# Using Predictive Analytics to Predict Delays in Maritime Container Shipping

In [1]:
%matplotlib inline

**Import Libraries**

In [2]:
import csv
import pandas as pd
import numpy as np
import pylab 
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

#http://pythondata.com/dask-large-csv-python/
#import dask.dataframe as dd
 
#from collections import Counter
#from patsy import dmatrices
#import statsmodels.api as sm

#https://www.dataquest.io/blog/pandas-big-data/

**Read in file**

In [3]:
df = pd.read_pickle('2014_data_top10.pkl')

In [4]:
#check that all rows imported
df.shape

(8003697, 24)

In [5]:
#check datatypes
df.dtypes

identifier                             uint64
trade_update_date              datetime64[ns]
run_date                       datetime64[ns]
vessel_name                          category
port_of_unlading                     category
estimated_arrival_date         datetime64[ns]
foreign_port_of_lading               category
record_status_indicator              category
place_of_receipt                     category
port_of_destination                  category
foreign_port_of_destination          category
actual_arrival_date            datetime64[ns]
consignee_name                       category
shipper_party_name                   category
container_number                     category
description_sequence_number            uint64
piece_count                            uint64
description_text                     category
carrier                              category
day_of_week_est                         uint8
day_of_week_act                         uint8
month_est                         

In [6]:
df[['identifier','trade_update_date','run_date','foreign_port_of_lading','port_of_unlading','estimated_arrival_date','actual_arrival_date']].iloc[0:50,:]

,identifier,trade_update_date,run_date,foreign_port_of_lading,port_of_unlading,estimated_arrival_date,actual_arrival_date
0,2014082158845,2014-07-30,2014-08-21,"Pusan,South Korea","Seattle, Washington",2014-08-14,2014-08-19
1,2014082158905,2014-07-30,2014-08-21,"Pusan,South Korea","Seattle, Washington",2014-08-14,2014-08-19
2,2014082158905,2014-07-30,2014-08-21,"Pusan,South Korea","Seattle, Washington",2014-08-14,2014-08-19
3,2014082158905,2014-07-30,2014-08-21,"Pusan,South Korea","Seattle, Washington",2014-08-14,2014-08-19
4,2014082158983,2014-08-06,2014-08-21,"Yantian,China (Mainland)","Long Beach, California",2014-08-19,2014-08-20
5,2014082159121,2014-07-30,2014-08-21,"Kaohsiung,China (Taiwan)","Tacoma, Washington",2014-08-18,2014-08-20
6,2014082159129,2014-07-30,2014-08-21,"Kaohsiung,China (Taiwan)","Tacoma, Washington",2014-08-18,2014-08-20
7,2014082159222,2014-07-30,2014-08-21,"Xiamen,China (Mainland)","Los Angeles, California",2014-08-19,2014-08-19
8,2014082159239,2014-07-30,2014-08-21,"Xiamen,China (Mainland)","Los Angeles, California",2014-08-19,2014-08-19
9,2014082159239,2014-07-30,2014-08-21,"Xiamen,China (Mainland)","Los Angeles, California",2014-08-19,2014-08-19


** Features to Consider **

In [7]:
df.head()

,identifier,trade_update_date,run_date,vessel_name,port_of_unlading,estimated_arrival_date,foreign_port_of_lading,record_status_indicator,place_of_receipt,port_of_destination,...,container_number,description_sequence_number,piece_count,description_text,carrier,day_of_week_est,day_of_week_act,month_est,month_act,delay_days
0,2014082158845,2014-07-30,2014-08-21,HYUNDAI FORCE,"Seattle, Washington",2014-08-14,"Pusan,South Korea",New,"XINGANG, PC",NaN,...,TGHU8036820,1,7881,IKEA HOME FURNISHING PRODUCTS TARIFF NUMBER 44...,HYUNDAI,3,1,8,8,5
1,2014082158905,2014-07-30,2014-08-21,HYUNDAI FORCE,"Seattle, Washington",2014-08-14,"Pusan,South Korea",New,"XINGANG, PC",NaN,...,APZU3172532,1,80,KLA-GARD PACKED ON 60 PALLETS -DRILLING MUD AD...,HYUNDAI,3,1,8,8,5
2,2014082158905,2014-07-30,2014-08-21,HYUNDAI FORCE,"Seattle, Washington",2014-08-14,"Pusan,South Korea",New,"XINGANG, PC",NaN,...,APZU3599329,1,80,KLA-GARD PACKED ON 60 PALLETS -DRILLING MUD AD...,HYUNDAI,3,1,8,8,5
3,2014082158905,2014-07-30,2014-08-21,HYUNDAI FORCE,"Seattle, Washington",2014-08-14,"Pusan,South Korea",New,"XINGANG, PC",NaN,...,TRLU3176120,1,80,KLA-GARD PACKED ON 60 PALLETS -DRILLING MUD AD...,HYUNDAI,3,1,8,8,5
4,2014082158983,2014-08-06,2014-08-21,OOCL SHENZHEN,"Long Beach, California",2014-08-19,"Yantian,China (Mainland)",New,"YANTIAN,CHINA",NaN,...,OOLU1064780,1,498,P.O.NO.050803/050938/051058/051197/051345/051 ...,OOCL,1,2,8,8,1


In [8]:
list(df)

['identifier',
 'trade_update_date',
 'run_date',
 'vessel_name',
 'port_of_unlading',
 'estimated_arrival_date',
 'foreign_port_of_lading',
 'record_status_indicator',
 'place_of_receipt',
 'port_of_destination',
 'foreign_port_of_destination',
 'actual_arrival_date',
 'consignee_name',
 'shipper_party_name',
 'container_number',
 'description_sequence_number',
 'piece_count',
 'description_text',
 'carrier',
 'day_of_week_est',
 'day_of_week_act',
 'month_est',
 'month_act',
 'delay_days']

** Features **  
vessel_name  
carrier (from vessel name)  
port_of_unlading  
foreign_port_of_lading  
estimated_arrival_date: day_of_week_est  
estimated_arrival_date: month_est  
record_status_indicator (?)  
place_of_receipt  
place_of_receipt vs foreign_port_of_lading  
consignee_name (check number of consignees (buyers) )  
shipper_party_name (check number of shipper)  
container_number: count of containers per identifier  
description_sequence_number (?)  
piece_count (sum over all containers)  
description_text (?)  
  
*features that aren't in the data set but would have been helpful: intermediate ports and number of intermediate ports,  planned length of trip, planned shipping route*  
  
** Target **  
delay_days (actual_arrival_date - estimated_arrival_date)  

In [9]:
len(df.identifier.unique())

3484891

In [10]:
df1 = df[['identifier','vessel_name','carrier','port_of_unlading','foreign_port_of_lading','day_of_week_est',
   'month_est','record_status_indicator','place_of_receipt','consignee_name','shipper_party_name',
   'container_number','description_sequence_number','piece_count','description_text','delay_days']]

In [18]:
df2 = df1[['identifier','container_number']].drop_duplicates()

In [19]:
df2.shape, df.shape

((5501913, 2), (8003697, 24))

In [20]:
#check duplicates - same identifier, same container number, same vessel name
df2 = df1[['identifier','container_number','vessel_name']].drop_duplicates()

In [21]:
df2.shape, df.shape

((5543353, 3), (8003697, 24))

In [28]:
#review the duplicates
df2_dup = df2[df2.duplicated(['identifier','container_number'],keep=False)].sort_values('identifier')
df2_dup.head(10)
#There are a lot of entries where the identifier and container_number are the same but the vessel_name is different,
#check the full record for the 2 identifiers listed below

,identifier,container_number,vessel_name
3141789,2014081255,MSKU0769198,MAERSK DENPASAR
3141776,2014081255,FSCU4823916,HYUNDAI TACOMA
3141772,2014081255,FSCU4823916,MAERSK DENPASAR
3141793,2014081255,MSKU0769198,HYUNDAI TACOMA
3141805,2014081255,PONU8081880,MAERSK DENPASAR
3141809,2014081255,PONU8081880,HYUNDAI TACOMA
2274381,2014081262,TRLU8770860,OOCL ITALY
2274377,2014081262,TRLU8770860,NYK TERRA
2274361,2014081262,OOLU7704420,NYK TERRA
2274359,2014081262,OOLU7704420,OOCL ITALY


In [33]:
#check if all the duplicates have short identifiers
df2_dup.identifier.unique().tolist()
#They don't, some are long

[2014081255L,
 2014081262L,
 2014081263L,
 2014081265L,
 2014081268L,
 2014081286L,
 2014081292L,
 20140812101L,
 20140812102L,
 20140812112L,
 20140812114L,
 20140812115L,
 20140812116L,
 20140812118L,
 20140812127L,
 20140812130L,
 20140812131L,
 20140812134L,
 20140812138L,
 20140812153L,
 20140812158L,
 20140812161L,
 20140812177L,
 20140812178L,
 20140812185L,
 20140812189L,
 20140812191L,
 20140812192L,
 20140812193L,
 20140812194L,
 20140812202L,
 20140812212L,
 20140812216L,
 20140812217L,
 20140812223L,
 20140812227L,
 20140812237L,
 20140812239L,
 20140812244L,
 20140812247L,
 20140812259L,
 20140812264L,
 20140812284L,
 20140812286L,
 20140812290L,
 20140812293L,
 20140812299L,
 20140812300L,
 20140812313L,
 20140812317L,
 20140812325L,
 20140812328L,
 20140812330L,
 20140812343L,
 20140812357L,
 20140812391L,
 20140812396L,
 20140812400L,
 20140812401L,
 20140812402L,
 20140812404L,
 20140812410L,
 20140812418L,
 20140812424L,
 20140812427L,
 20140812428L,
 20140812444L,
 2

In [31]:
#check records
df.iloc[:,:-5][df.identifier.isin([2014081262])]
#The ID is not unique and appears to be re-used.  


,identifier,trade_update_date,run_date,vessel_name,port_of_unlading,estimated_arrival_date,foreign_port_of_lading,record_status_indicator,place_of_receipt,port_of_destination,foreign_port_of_destination,actual_arrival_date,consignee_name,shipper_party_name,container_number,description_sequence_number,piece_count,description_text,carrier
2274359,2014081262,2014-02-21,2014-08-12,OOCL ITALY,"Los Angeles, California",2013-11-25,"Shanghai ,China (Mainland)",Amended,SHANGHAI,NaN,NaN,2013-11-25,NORTHDOWN INDUSTRIES,"MILOTEX (HANGZHOU)CO.,LTD",OOLU7704420,1,1200,"SILICA GEL CAT LITTER PO18542422 ""SHIPMENT C """,OOCL
2274361,2014081262,2013-11-05,2014-08-12,NYK TERRA,"Los Angeles, California",2014-03-16,"Tsingtao,China (Mainland)",Amended,"QINGDAO, CHINA",NaN,NaN,2014-03-19,"HELMET HOUSE, INC.",QINGDAO FRANK FINE CHEMICAL INDUSTR,OOLU7704420,1,1200,"SILICA GEL CAT LITTER PO18542422 ""SHIPMENT C """,NYK
2274362,2014081262,2013-11-05,2014-08-12,NYK TERRA,"Los Angeles, California",2014-03-16,"Tsingtao,China (Mainland)",Amended,"QINGDAO, CHINA",NaN,NaN,2014-03-19,NORTHDOWN INDUSTRIES,QINGDAO FRANK FINE CHEMICAL INDUSTR,OOLU7704420,1,1200,"SILICA GEL CAT LITTER PO18542422 ""SHIPMENT C """,NYK
2274363,2014081262,2014-02-21,2014-08-12,NYK TERRA,"Los Angeles, California",2014-03-16,"Tsingtao,China (Mainland)",Amended,"QINGDAO, CHINA",NaN,NaN,2014-03-19,"HELMET HOUSE, INC.",QINGDAO FRANK FINE CHEMICAL INDUSTR,OOLU7704420,1,1200,"SILICA GEL CAT LITTER PO18542422 ""SHIPMENT C """,NYK
2274364,2014081262,2014-02-21,2014-08-12,NYK TERRA,"Los Angeles, California",2014-03-16,"Tsingtao,China (Mainland)",Amended,"QINGDAO, CHINA",NaN,NaN,2014-03-19,NORTHDOWN INDUSTRIES,QINGDAO FRANK FINE CHEMICAL INDUSTR,OOLU7704420,1,1200,"SILICA GEL CAT LITTER PO18542422 ""SHIPMENT C """,NYK
2274365,2014081262,2013-11-05,2014-08-12,OOCL ITALY,"Los Angeles, California",2013-11-25,"Shanghai ,China (Mainland)",Amended,SHANGHAI,NaN,NaN,2013-11-25,"HELMET HOUSE, INC.",QINGDAO FRANK FINE CHEMICAL INDUSTR,OOLU7704420,1,1200,"SILICA GEL CAT LITTER PO18542422 ""SHIPMENT C """,OOCL
2274366,2014081262,2013-11-05,2014-08-12,OOCL ITALY,"Los Angeles, California",2013-11-25,"Shanghai ,China (Mainland)",Amended,SHANGHAI,NaN,NaN,2013-11-25,NORTHDOWN INDUSTRIES,QINGDAO FRANK FINE CHEMICAL INDUSTR,OOLU7704420,1,1200,"SILICA GEL CAT LITTER PO18542422 ""SHIPMENT C """,OOCL
2274367,2014081262,2014-02-21,2014-08-12,OOCL ITALY,"Los Angeles, California",2013-11-25,"Shanghai ,China (Mainland)",Amended,SHANGHAI,NaN,NaN,2013-11-25,"HELMET HOUSE, INC.",QINGDAO FRANK FINE CHEMICAL INDUSTR,OOLU7704420,1,1200,"SILICA GEL CAT LITTER PO18542422 ""SHIPMENT C """,OOCL
2274368,2014081262,2014-02-21,2014-08-12,OOCL ITALY,"Los Angeles, California",2013-11-25,"Shanghai ,China (Mainland)",Amended,SHANGHAI,NaN,NaN,2013-11-25,NORTHDOWN INDUSTRIES,QINGDAO FRANK FINE CHEMICAL INDUSTR,OOLU7704420,1,1200,"SILICA GEL CAT LITTER PO18542422 ""SHIPMENT C """,OOCL
2274369,2014081262,2013-11-05,2014-08-12,NYK TERRA,"Los Angeles, California",2014-03-16,"Tsingtao,China (Mainland)",Amended,"QINGDAO, CHINA",NaN,NaN,2014-03-19,"HELMET HOUSE, INC.","MILOTEX (HANGZHOU)CO.,LTD",OOLU7704420,1,1200,"SILICA GEL CAT LITTER PO18542422 ""SHIPMENT C """,NYK


Identifier 2014081262 contains information for the same container for two completely different set of dates,
on different vessels.    
1- The run_date is the same for all, but the trade_update_date, estimated and actual arrival dates are different for the two sets  
2 - there are multiple shippers

Conclusion: to get unique ship information use: vessel_name and actual_arrival_date

In [35]:
#check vessel_name and actual_arrival_date unique values 
#then add ports and estimated arrival dates in and check that the number of unique observations doen't change
df3 = df[['vessel_name','actual_arrival_date']].drop_duplicates()
df3.shape

(29846, 2)

In [43]:
#look at vessel, arrival data, and port of lading and unlading
df4 = df[['vessel_name','actual_arrival_date','foreign_port_of_lading','port_of_unlading']].drop_duplicates()
df4.shape
#why do adding ports increase the number of observations? 
#how can the same vessel arrive on the same date to and from different ports?
#It can if the same ship has multiple stops, and different containers are loaded at different points
        #How much information does this provide about intermediate stops...
        #Cargo that was loaded at a non-US port and then unloaded at another non-US port would not have any record
        #AMS system, so this wouldn't necessarily provide all stops

(99944, 4)

In [46]:
#look at the duplicate values
df4[df4.duplicated(['vessel_name','actual_arrival_date'], keep = False)].sort_values(
    ['vessel_name','actual_arrival_date']).head(10)

,vessel_name,actual_arrival_date,foreign_port_of_lading,port_of_unlading
736360,9077276,2014-04-01,"Yokohama,Japan","Los Angeles, California"
9178662,9077276,2014-04-01,"Tokyo ,Japan","Los Angeles, California"
496002,9143037,2014-06-02,"Singapore,Singapore","Seattle, Washington"
4498700,9143037,2014-06-02,"Yokohama,Japan","Los Angeles, California"
1925337,9200691,2014-06-03,"Thanh Pho Ho Chi Minh,Vietnam","Long Beach, California"
3655666,9200691,2014-06-03,"Port Swettenham,Malaysia","Long Beach, California"
1218462,9243186,2014-11-26,"Southhampton,United Kingdom","Houston, Texas"
10233585,9243186,2014-11-26,"Anvers,Belgium","Houston, Texas"
5950209,9243203,2014-01-15,"Southhampton,United Kingdom","Houston, Texas"
6519166,9243203,2014-01-15,"Bremerhaven,Federal Republic of Germany","Houston, Texas"


In [53]:
#Look at observations for these vessel and arrival data combos
df[(df.vessel_name == '9077276') & (df.actual_arrival_date == '2014-04-01')].iloc[:,:-5].sort_values('actual_arrival_date')

,identifier,trade_update_date,run_date,vessel_name,port_of_unlading,estimated_arrival_date,foreign_port_of_lading,record_status_indicator,place_of_receipt,port_of_destination,foreign_port_of_destination,actual_arrival_date,consignee_name,shipper_party_name,container_number,description_sequence_number,piece_count,description_text,carrier
736360,2014040245893,2014-03-19,2014-04-02,9077276,"Los Angeles, California",2014-03-21,"Yokohama,Japan",New,YOKOHAMA,NaN,NaN,2014-04-01,"TOYO DENKI USA, INC",TOYO DENKI SEIZO K.K.,TEMU9194833,1,1,PROPULSION INVERTER PARTS HS CODE: 8535.90,9077276
4456318,2014040245889,2014-03-19,2014-04-02,9077276,"Los Angeles, California",2014-03-21,"Yokohama,Japan",New,YOKOHAMA,NaN,NaN,2014-04-01,DENGENSHA AMERICA CORP.,"DENGENSHA MFG. CO., LTD.",TEMU9194833,1,3,WELDER HS CODE: 8515.21,9077276
7739254,2014040245892,2014-03-19,2014-04-02,9077276,"Los Angeles, California",2014-03-21,"Yokohama,Japan",New,YOKOHAMA,NaN,NaN,2014-04-01,NaN,NaN,TEMU9194833,1,154,AUTO SPARE PARTS DISK BRAKE PAD HS CODE: 6813.81,9077276
9178662,2014040245882,2014-03-19,2014-04-02,9077276,"Los Angeles, California",2014-03-21,"Tokyo ,Japan",New,TOKYO,NaN,NaN,2014-04-01,HOFFMAN BROTHERS,"SEIKO SEWING MACHINE CO., LTD.",MORU1300102,1,1,"""SEIKO"" BRAND INDUSTRIAL SEWING MACHINE HS COD...",9077276


In [51]:
#Look at observations for these vessel and arrival data combos
df[(df.vessel_name == '9243186') & (df.actual_arrival_date == '2014-11-26')].iloc[:,:-5].sort_values('actual_arrival_date')

,identifier,trade_update_date,run_date,vessel_name,port_of_unlading,estimated_arrival_date,foreign_port_of_lading,record_status_indicator,place_of_receipt,port_of_destination,foreign_port_of_destination,actual_arrival_date,consignee_name,shipper_party_name,container_number,description_sequence_number,piece_count,description_text,carrier
1218462,2014112745388,2014-11-07,2014-11-27,9243186,"Houston, Texas",2014-11-25,"Southhampton,United Kingdom",New,UNITED KINGDOM,NaN,NaN,2014-11-26,JAROPAMEX ACUNA,WARDLE STOREYS,HLXU8442016,1,2,PVC SHEETING,9243186
10233585,2014112750078,2014-11-13,2014-11-27,9243186,"Houston, Texas",2014-11-27,"Anvers,Belgium",New,THE NETHERLANDS,NaN,NaN,2014-11-26,PACIFIC DRILLING SERVICES INC,PACIFIC DRILLING SERVICES INC,HLXU3623834,1,3,OILWELL DRILLING EQUIPMENT,9243186


In [54]:
#look at foreign port of unlading - more than one origin for the ship
df5 = df[['vessel_name','actual_arrival_date','foreign_port_of_lading']].drop_duplicates()
df5.shape, df3.shape

((92143, 3), (29846, 2))

In [57]:
#look at foreign port of unlading - more than one origin for the ship
df6 = df[['vessel_name','actual_arrival_date','port_of_unlading']].drop_duplicates()
df6.shape, df3.shape

((34671, 3), (29846, 2))

In [60]:
#look at the duplicate values
df6[df6.duplicated(['vessel_name','actual_arrival_date'], keep = False)].sort_values(
    ['vessel_name','actual_arrival_date'])
#how can a vessel arrive in two different ports a thousand miles apart on the same day???
#this must be the arrival at the final destination
        # which estimated arrival date should be used to calculate delay_days??? 

,vessel_name,actual_arrival_date,port_of_unlading
496002,9143037,2014-06-02,"Seattle, Washington"
4498700,9143037,2014-06-02,"Los Angeles, California"
3195364,9364992,2014-10-20,"Long Beach, California"
8766477,9364992,2014-10-20,"Los Angeles, California"
8231124,9401178,2014-12-24,"Charleston, South Carolina"
9961833,9401178,2014-12-24,"Houston, Texas"
1084085,9695016,2014-07-02,"Los Angeles, California"
6365127,9695016,2014-07-02,"Long Beach, California"
105352,ADELHEID-S.,2014-01-29,"Los Angeles, California"
5681304,ADELHEID-S.,2014-01-29,"Long Beach, California"


In [59]:
#Look at observations for these vessel and arrival data combos
df[(df.vessel_name == '9143037') & (df.actual_arrival_date == '2014-06-02')].iloc[:,:-5].sort_values('actual_arrival_date')

,identifier,trade_update_date,run_date,vessel_name,port_of_unlading,estimated_arrival_date,foreign_port_of_lading,record_status_indicator,place_of_receipt,port_of_destination,foreign_port_of_destination,actual_arrival_date,consignee_name,shipper_party_name,container_number,description_sequence_number,piece_count,description_text,carrier
496002,2014060313484,2014-05-20,2014-06-03,9143037,"Seattle, Washington",2014-05-30,"Singapore,Singapore",New,SINGAPORE SG,NaN,NaN,2014-06-02,"BLUESTEM BRANDS, INC.","ROYCE ENTERPRISE CO., LTD",MSKU2270021,1,175,4QQ6190000010 MYHOME 6 DRAWER DRESSER,9143037
496003,2014060313484,2014-05-20,2014-06-03,9143037,"Seattle, Washington",2014-05-30,"Singapore,Singapore",New,SINGAPORE SG,NaN,NaN,2014-06-02,"BLUESTEM BRANDS, INC.","ROYCE ENTERPRISE CO., LTD",MSKU2270021,2,200,4QQ6200000010 MYHOME 2 DRAWER NIGHTSTAND,9143037
496005,2014060313484,2014-05-20,2014-06-03,9143037,"Seattle, Washington",2014-05-30,"Singapore,Singapore",New,SINGAPORE SG,NaN,NaN,2014-06-02,"BLUESTEM BRANDS, INC.","ROYCE ENTERPRISE CO., LTD",TCNU8157049,1,200,4QQ6150000010 MYHOME 1 DRAWER NIGHT STAND,9143037
496006,2014060313484,2014-05-20,2014-06-03,9143037,"Seattle, Washington",2014-05-30,"Singapore,Singapore",New,SINGAPORE SG,NaN,NaN,2014-06-02,"BLUESTEM BRANDS, INC.","ROYCE ENTERPRISE CO., LTD",TCNU8157049,2,200,4QQ6160000010 MYHOME 4 DRAWER CHEST,9143037
496007,2014060313484,2014-05-20,2014-06-03,9143037,"Seattle, Washington",2014-05-30,"Singapore,Singapore",New,SINGAPORE SG,NaN,NaN,2014-06-02,"BLUESTEM BRANDS, INC.","ROYCE ENTERPRISE CO., LTD",TCNU8157049,3,200,4QQ6180000010 MYHOME 3 3 DRAWER CHEST,9143037
496008,2014060313484,2014-05-20,2014-06-03,9143037,"Seattle, Washington",2014-05-30,"Singapore,Singapore",New,SINGAPORE SG,NaN,NaN,2014-06-02,"BLUESTEM BRANDS, INC.","ROYCE ENTERPRISE CO., LTD",TCNU8157049,4,25,4QQ6190000010 MYHOME 6 DRAWER DRESSER,9143037
4498700,2014060362188,2014-05-20,2014-06-03,9143037,"Los Angeles, California",2014-06-02,"Yokohama,Japan",New,TOKYO,NaN,NaN,2014-06-02,PARTSMART,"M.F.TRADING CO.,LTD.",MWCU5286546,1,25,LASER PRINTER PARTS,9143037
8921986,2014060329777,2014-05-12,2014-06-03,9143037,"Los Angeles, California",2014-06-02,"Yokohama,Japan",New,QINGDAO,NaN,NaN,2014-06-02,NaN,NaN,CMAU8294605,1,1742,COMMUNICATION CABLE INSULATED WIRE AND CABLE,9143037
9504910,2014060362140,2014-05-20,2014-06-03,9143037,"Los Angeles, California",2014-06-02,"Yokohama,Japan",New,TOKYO,NaN,NaN,2014-06-02,AKEBONO BRAKE COLOMBIA PLANT,NOK CORPORATION,MWCU5286546,1,80,"""NOK"" BRAND PRODUCTS",9143037


In [63]:
#Compare number of unique actual_arrival_date + vessel to run_date + vessel
df7 = df[['vessel_name','run_date']].drop_duplicates()
df7.shape, df3.shape

((65232, 2), (29846, 2))

In [71]:
df7[df7.duplicated(['vessel_name'])].sort_values('vessel_name').head(10)

,vessel_name,run_date
6511767,8711368,2014-01-06
4326955,9143037,2014-10-07
3900776,9243162,2014-04-11
7952408,9243174,2014-01-01
1590617,9243186,2014-03-29
7382819,9243186,2014-01-13
10299929,9243186,2014-02-22
8074406,9243203,2014-06-13
5950209,9243203,2014-01-16
8817529,9262728,2014-01-18


In [73]:
df[df.vessel_name == '8711368'].iloc[:,:-5]

,identifier,trade_update_date,run_date,vessel_name,port_of_unlading,estimated_arrival_date,foreign_port_of_lading,record_status_indicator,place_of_receipt,port_of_destination,foreign_port_of_destination,actual_arrival_date,consignee_name,shipper_party_name,container_number,description_sequence_number,piece_count,description_text,carrier
667434,201401043134,2013-12-09,2014-01-04,8711368,"Houston, Texas",2014-01-01,"Bremerhaven,Federal Republic of Germany",New,SHEFFIELD,NaN,NaN,2014-01-03,VESTAS NACELLES AMERICA INC.,COOPER & TURNER LTD,FSCU6490223,1,2,WINDMILL PARTS,8711368
6227593,201401043135,2013-12-09,2014-01-04,8711368,"Houston, Texas",2014-01-01,"Bremerhaven,Federal Republic of Germany",New,WERLTE,NaN,NaN,2014-01-03,VESTAS NACELLES AMERICA INC.,COMPONENTA B.V.,FSCU6490223,1,14,WINDMILL PARTS,8711368
6511767,201401063236,2013-12-09,2014-01-06,8711368,"Houston, Texas",2014-01-01,"Bremerhaven,Federal Republic of Germany",Amended,WERLTE,NaN,NaN,2014-01-03,VESTAS NACELLES AMERICA INC.,COMPONENTA B.V.,FSCU6490223,1,14,WINDMILL PARTS,8711368


** Estimated and Actual Arrival Dates **
The data set appears to only provide the actual_arrival_date for the final U.S. port of unlading   
  
For the purpose of this analysis:  
- The unit of analysis will be the ship run, and all interim stops will be assigned the same delay as the general ship  
How to group by ship run:    
   
PROBLEM  
After the fact data can be grouped by vessel name and actual arrival date since only one ship can arrive in one place at one time.   
Can't group on actual_arrival for the analysis because that won't be known when implementing the model to predict delays.  
    The identifier is not unique to a ship or shipment  
    The run_date can vary as can the trade_update_date  
    The ports can vary (multiple stops)     
  
POSSIBLE SOLUTION  
Get the first estimated arrival date for each ship     
        then include all estimated arrival dates that are +/- 15? days as connected to that ship's run 
        since this is international shipping it is expected that most ships won't reappear in the data for a month  
        or so on a new run.  test the number of days against grouping by actual arrival + vessel_name  
  
RIGOUROUS METHOD:  
- Sort data set by vessel_name and then by estimated arrival date  
- Enter a ship run index of 1 for all entries for all ships (combined with the ship name this will become the ship run number)   
- For each ship compare each subsequent estimated arrival to the first arrival, if it's within 15 days of the previous estimated arrival date, don't change the index, if it's not increase the index by 1, continue for the rest of the observations for that ship and then for all ships in the data set  
- For each ship run identify the maximum estimated arrival date and the delay_days between that max and the actual_arrival date, create a max estimated arrival date column and enter the max for that ship run in all observations for that ship run.
- For each ship run, get the number of unique ports of lading and unlading, and use at the total (known) port stops for that run, put this in a new column (this overestimates stops for interim pick up and drop off locations)
  
SIMPLIFIED METHOD:  (would have to use the above for true prediction)
- use the vessel_name + actual_arrival_date to identify vessel runs, and calculate the max estimated_arrival_date for each, the delay_days based on this max, and the number of unique ports of lading and unlading
- add a column and for each vessel and number the observations by actual_arrival_date to create vessel run numbers  
- Bring all of these fields back into the main dataset

In [74]:
df_vessels = df[['vessel_name','actual_arrival_date','estimated_arrival_date',
                 'foreign_port_of_lading','port_of_unlading']]

In [75]:
df_vessels.head(10)

,vessel_name,actual_arrival_date,estimated_arrival_date,foreign_port_of_lading,port_of_unlading
0,HYUNDAI FORCE,2014-08-19,2014-08-14,"Pusan,South Korea","Seattle, Washington"
1,HYUNDAI FORCE,2014-08-19,2014-08-14,"Pusan,South Korea","Seattle, Washington"
2,HYUNDAI FORCE,2014-08-19,2014-08-14,"Pusan,South Korea","Seattle, Washington"
3,HYUNDAI FORCE,2014-08-19,2014-08-14,"Pusan,South Korea","Seattle, Washington"
4,OOCL SHENZHEN,2014-08-20,2014-08-19,"Yantian,China (Mainland)","Long Beach, California"
5,HYUNDAI GLOBAL,2014-08-20,2014-08-18,"Kaohsiung,China (Taiwan)","Tacoma, Washington"
6,HYUNDAI GLOBAL,2014-08-20,2014-08-18,"Kaohsiung,China (Taiwan)","Tacoma, Washington"
7,COSCO HARMONY,2014-08-19,2014-08-19,"Xiamen,China (Mainland)","Los Angeles, California"
8,COSCO HARMONY,2014-08-19,2014-08-19,"Xiamen,China (Mainland)","Los Angeles, California"
9,COSCO HARMONY,2014-08-19,2014-08-19,"Xiamen,China (Mainland)","Los Angeles, California"


In [83]:
df[df.vessel_name == 'ZIM VIRGINIA'].sort_values('actual_arrival_date').actual_arrival_date.unique()

array(['2013-12-31T00:00:00.000000000', '2014-01-04T00:00:00.000000000',
       '2014-01-05T00:00:00.000000000', '2014-01-06T00:00:00.000000000',
       '2014-01-07T00:00:00.000000000', '2014-01-08T00:00:00.000000000',
       '2014-01-14T00:00:00.000000000', '2014-01-15T00:00:00.000000000',
       '2014-01-16T00:00:00.000000000', '2014-01-17T00:00:00.000000000',
       '2014-01-20T00:00:00.000000000', '2014-03-05T00:00:00.000000000',
       '2014-03-10T00:00:00.000000000', '2014-03-11T00:00:00.000000000',
       '2014-03-19T00:00:00.000000000', '2014-03-20T00:00:00.000000000',
       '2014-03-21T00:00:00.000000000', '2014-03-23T00:00:00.000000000',
       '2014-05-29T00:00:00.000000000', '2014-05-30T00:00:00.000000000',
       '2014-05-31T00:00:00.000000000', '2014-06-01T00:00:00.000000000',
       '2014-06-02T00:00:00.000000000', '2014-06-04T00:00:00.000000000',
       '2014-06-05T00:00:00.000000000', '2014-08-03T00:00:00.000000000',
       '2014-08-10T00:00:00.000000000', '2014-08-14

In [96]:
#check the Zim Virginia as it seems to have some strange data,
df.iloc[:,:-5][df.vessel_name == 'ZIM VIRGINIA'].sort_values('actual_arrival_date')[['vessel_name','actual_arrival_date','estimated_arrival_date','foreign_port_of_lading','port_of_unlading']].drop_duplicates()[30:70]
#the Zim Virginia does have different arrival dates by port

,vessel_name,actual_arrival_date,estimated_arrival_date,foreign_port_of_lading,port_of_unlading
382735,ZIM VIRGINIA,2014-01-06,2013-12-31,"Kaohsiung,China (Taiwan)","Charleston, South Carolina"
8517926,ZIM VIRGINIA,2014-01-06,2014-01-05,"Puerto Manzanillo,Panama","Charleston, South Carolina"
3829793,ZIM VIRGINIA,2014-01-06,2013-12-31,"Pusan,South Korea","Charleston, South Carolina"
3829713,ZIM VIRGINIA,2014-01-06,2013-12-24,"Tsingtao,China (Mainland)","Charleston, South Carolina"
8807933,ZIM VIRGINIA,2014-01-06,2014-01-04,"Port Bustamante,Jamaica","Charleston, South Carolina"
8808318,ZIM VIRGINIA,2014-01-06,2013-12-17,"Tsingtao,China (Mainland)","Charleston, South Carolina"
8808642,ZIM VIRGINIA,2014-01-06,2013-12-31,"Shanghai ,China (Mainland)","Charleston, South Carolina"
5666196,ZIM VIRGINIA,2014-01-06,2013-12-31,"Xiamen,China (Mainland)","Charleston, South Carolina"
1247322,ZIM VIRGINIA,2014-01-06,2014-01-05,"Port Bustamante,Jamaica","Charleston, South Carolina"
668548,ZIM VIRGINIA,2014-01-06,2013-12-30,"Port Bustamante,Jamaica","Charleston, South Carolina"


** Limit to just the top US port and possible just one of its trading ports **

Explore the data between one set of ports